In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel
import transformers
import os
import torch

device = "cuda"

FIM_PREFIX = "<fim-prefix>"
FIM_MIDDLE = "<fim-middle>"
FIM_SUFFIX = "<fim-suffix>"
FIM_PAD = "<fim-pad>"
EOD = "<|endoftext|>"

tokenizer = AutoTokenizer.from_pretrained("bigcode/santacoder", padding_side="left")
tokenizer.add_special_tokens(
    {
        "additional_special_tokens": [EOD, FIM_PREFIX, FIM_MIDDLE, FIM_SUFFIX, FIM_PAD],
        "eos_token": EOD,
        "pad_token": FIM_PAD,
    }
)
tokenizer.pad_token_id


49156

In [2]:
model: GPT2LMHeadModel = AutoModelForCausalLM.from_pretrained(
    "bigcode/santacoder", revision="aaeed52", trust_remote_code=True
).to(device)


In [3]:
import re

sample_code = """
prompts = []
suggestion_index_to_token_index = []

for i, token in enumerate(tokens):
    if token.isspace() == "":
        # Don't generate suggestions for whitespace
        continue
    else:
        prefix = ''.join(tokens[:i])
        suffix = ''.join(tokens[i + 1 :])
        prompt = FIM_PREFIX + prefix + FIM_SUFFIX + suffix + FIM_MIDDLE

        prompts.append(prompt)
        suggestion_index_to_token_index.append(i)"""

# Split the code into tokens naively on whitespace
tokens = re.split(r"(\s+)", sample_code)

len(tokens)


91

In [9]:
prompts = []
suggestion_index_to_token_index = []

for i, token in enumerate(tokens):
    if token.isspace() == "":
        # Don't generate suggestions for whitespace
        continue
    else:
        prefix = ''.join(tokens[:i])
        suffix = ''.join(tokens[i + 1 :])
        prompt = FIM_PREFIX + prefix + FIM_SUFFIX + suffix + FIM_MIDDLE

        prompts.append(prompt)
        suggestion_index_to_token_index.append(i)

<fim-prefix>
prompts = []
suggestion_index_to_token_index = []

for i, token in enumerate(tokens):
    if token.isspace() == <fim-suffix>
        # Don't generate suggestions for whitespace
        continue
    else:
        prefix = ''.join(tokens[:i])
        suffix = ''.join(tokens[i + 1 :])
        prompt = FIM_PREFIX + prefix + FIM_SUFFIX + suffix + FIM_MIDDLE

        prompts.append(prompt)
        suggestion_index_to_token_index.append(i)<fim-middle>


In [5]:
@torch.no_grad()
def get_suggestions(input_prompts):
    outputs_per_input = 5

    inputs = tokenizer(
        input_prompts, return_tensors="pt", padding=True, return_token_type_ids=False
    ).to(device)

    outputs = model.generate(
        **inputs,
        # Only allow the model to generate up to 10 tokens. If we
        # need more than that to fix your code it's not a typo anymore!
        max_new_tokens=20,
        num_return_sequences=outputs_per_input,
        num_beams=outputs_per_input,
        early_stopping=True,
        output_scores=True,
        return_dict_in_generate=True,
        # Always force the model to generate the eos token. If an EOS is improbable, that's very important information!
        forced_eos_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    # Adapted from https://discuss.huggingface.co/t/generation-probabilities-how-to-compute-probabilities-of-output-scores-for-gpt2/3175
    # Just the generated sequences, ignoring the prompt
    gen_sequences = outputs.sequences[:, inputs.input_ids.shape[-1] :]

    # For each sequence, get the probability of each token at each step
    probs = torch.stack(outputs.scores, dim=1).softmax(-1)
    
    gen_probs = torch.gather(probs, 2, gen_sequences[:, :, None]).squeeze(-1)

    # Wherever the sequence is the pad token, set the probability to 1. That way we don't penalize short sequences. We already penalize premature truncation by forcing the model to generate the EOS token.
    gen_probs = torch.where(
        gen_sequences == tokenizer.pad_token_id,
        torch.ones_like(gen_probs),
        gen_probs,
    )

    # Multiply all the probabilities together to get the probability of the entire sequence being generated
    unique_prob_per_sequence = gen_probs.prod(-1)
    
    # Decode the returned sequences into strings
    sequences = tokenizer.batch_decode(gen_sequences, skip_special_tokens=True)

    sequences_with_probs = list(zip(sequences, unique_prob_per_sequence.tolist()))

    all_results = []

    for i in range(len(input_prompts)):
        batch_sequences = sequences_with_probs[i * outputs_per_input : (i + 1) * outputs_per_input]
        
        batch_sequences = sorted(batch_sequences, key=lambda x: x[1], reverse=True)
        all_results.append(batch_sequences)
        
    return all_results


get_suggestions(prompts[0:3])

[[('\nFIM_PREFIX = "<s>"\nFIM_SUFFIX = "</s>"\n', 0.0),
  ('\nFIM_PREFIX = "<FIM>"\nFIM_SUFFIX = "</FIM', 0.0),
  ('\nFIM_PREFIX = "<FIM>"\nFIM_SUFFIX = "<FIM', 0.0),
  ('\nFIM_PREFIX = "FIM: "\nFIM_SUFFIX = " "', 0.0),
  ('\nFIM_PREFIX = "FIM: "\nFIM_MIDDLE = " "', 0.0)],
 [('=', 0.9707012176513672),
  ('= ', 0.00410509156063199),
  ('+=', 0.0005870978930033743),
  ('= []\nsuggestion_tokens =', 1.3415783241266244e-12),
  ('= []\nsuggestions =', 2.3447771211285294e-25)],
 [('[]', 0.6721534132957458),
  ('[]\n', 0.041928596794605255),
  ('list()', 0.01677822507917881),
  ('[', 0.00012675553443841636),
  ('[""]', 1.106857010560458e-12)]]

In [6]:

model_outputs = get_suggestions(prompts)

# Create an array of the samen length as `prompts`, full of `None`
suggestions = [None] * len(tokens)

for token_index, suggestion in zip(suggestion_index_to_token_index, model_outputs):
    suggestions[token_index] = suggestion

In [7]:
from IPython.display import HTML

css = """
#output {
  font-family: monospace;
  white-space: pre;
  background-color: #333;
  padding: 20px;
  padding-bottom: 140px;
}

.token {
  color: #000;
  position: relative;
  cursor: pointer;
}

.alternatives {
  position: absolute;
  display: none;
}

.token:hover {
  background-color: #ddd;
}

.token:hover .alternatives {
  color: #000;
  position: absolute;
  top: 14px;
  left: 0;
  display: block;
  background-color: #fff;
  z-index: 1;
  padding: 8px;
  border-radius: 8px;
  border: 1px solid #ccc;
}
"""

spans = []
for token, suggestion in zip(tokens, suggestions):
    if suggestion is None:
        spans.append(f"<span class='token'>{token}</span>")
        continue

    # Find the token in the suggestions list and determine how likely it is
    token_probability = 0
    for s, p in suggestion:
        if s == token:
            token_probability = p
            break

    top_probability = suggestion[0][1]

    token_color = "white"
    if top_probability > 0.5 and token_probability < 0.1:
        token_color = "red"
    
    # token_color = f"hsl({(token_probability) * 120}, 100%, 50%)"

    suggestions_items = []
    for s, p in suggestion:
        if s == token:
            s = f"<strong>{s}</strong>"
        if s == "":
            s = "<i>remove token</i>"

        suggestions_items.append(f"<tr><td>{p:.2f}</td><td>{s}</td></li>")

    suggestions_table = f"<table>{''.join(suggestions_items)}</table>"

    spans.append(
        f"<span class='token' style='color: {token_color}'>{token}<span class='alternatives'>{suggestions_table}</span></span>"
    )

display(
    HTML(f"<style>{css}</style><body><div id='output'>{''.join(spans)}</div></body>")
)


0.00,"FIM_PREFIX = """" FIM_SUFFIX = """""
0.00,"FIM_PREFIX = """" FIM_SUFFIX = ""</FIM"
0.00,"FIM_PREFIX = """" FIM_SUFFIX = ""<FIM"
0.00,"FIM_PREFIX = ""FIM: "" FIM_SUFFIX = "" """
0.00,"FIM_PREFIX = ""FIM: "" FIM_MIDDLE = "" """
0.97,=
0.00,=
0.00,+=
0.00,= [] suggestion_tokens =
0.00,= [] suggestions =
0.67,[]
